# Iterables & Iterators — Practice (Advanced, but not too much)

What you’ll practice:
- Distinguishing **iterables** vs **iterators**
- Safely consuming from iterators (one-shot) and from re-iterable iterables
- Chunking, sliding windows, and round-robin across iterables
- Unique filtering that also works with unhashable elements
- Flattening nested structures without exploding strings
- Building small iterator utilities: `peek`, `seek`, and a rewindable wrapper

Each task has a TODO cell and asserts. Run the asserts to verify.

### Helpers (used in multiple tasks)

In [1]:
def is_iterable(obj):
    """Return True if obj can produce an iterator via iter(obj)."""
    try:
        iter(obj)
        return True
    except TypeError:
        return False

def is_iterator(obj):
    """Return True if obj is its own iterator (one-shot)."""
    try:
        return iter(obj) is obj
    except TypeError:
        return False

## 1) Classify objects: iterable / iterator / non-iterable
Implement `type_of_iteration(obj)` returning one of: `'iterable'`, `'iterator'`, `'non-iterable'`.
- Use EAFP: try `iter(obj)` and check `iter(obj) is obj` for iterators.

In [2]:
# TODO: implement type_of_iteration
def type_of_iteration(obj):
    try:
        it = iter(obj)
    except TypeError:
        return 'non-iterable'
    else:
        return 'iterator' if it is obj else 'iterable'

type_of_iteration([1,2,3])

'iterable'

In [3]:
assert type_of_iteration([1,2]) == 'iterable'
assert type_of_iteration(iter([1,2])) == 'iterator'
assert type_of_iteration((x for x in range(3))) == 'iterator'
assert type_of_iteration(10) == 'non-iterable'
print('OK - 1')

OK - 1


## 2) Take first *n* elements without over-consuming
Implement `first_n(iterable, n)` that returns a list of up to `n` elements.
- If given an **iterator**, it should consume exactly the yielded items.
- If given a **re-iterable** (e.g., list/range), it should **not** modify it (normal behavior).
- Don’t convert the entire input to a list.

In [4]:
# TODO: implement first_n
def first_n(iterable, n):
    it = iter(iterable)
    out = []
    for _ in range(max(0, int(n))):
        try:
            out.append(next(it))
        except StopIteration:
            break
    return out

first_n(range(10), 3)

[0, 1, 2]

In [5]:
assert first_n([10,20,30], 2) == [10,20]
it = iter([1,2,3,4])
assert first_n(it, 2) == [1,2]
assert next(it) == 3  # ensure we consumed exactly 2
print('OK - 2')

OK - 2


## 3) Chunk an iterable into fixed-size blocks
Implement `chunk(iterable, size)` yielding tuples of length `size` (last may be shorter).
- Don’t materialize the whole input.
- Works with both iterables and iterators.

In [6]:
# TODO: implement chunk
def chunk(iterable, size):
    if size <= 0:
        raise ValueError('size must be > 0')
    it = iter(iterable)
    while True:
        batch = []
        for _ in range(size):
            try:
                batch.append(next(it))
            except StopIteration:
                break
        if not batch:
            break
        yield tuple(batch)

list(chunk(range(7), 3))

[(0, 1, 2), (3, 4, 5), (6,)]

In [7]:
assert list(chunk([1,2,3,4,5], 2)) == [(1,2),(3,4),(5,)]
it = (i for i in range(5))
assert list(chunk(it, 3)) == [(0,1,2),(3,4)]
print('OK - 3')

OK - 3


## 4) Unique (ever-seen) that tolerates unhashable values
Implement `unique_everseen(iterable, key=None)` yielding first occurrences while preserving order.
- If items (or keys) are hashable, use a set.
- If not hashable (e.g., lists), fall back to a list check.
- Optional `key(x)` can transform elements for comparison.

In [8]:
def unique_everseen(iterable, key=None):
    seen_hashable = set()      # for hashable originals OR key results (when unhashable originals)
    seen_unhashable = []       # fallback list for unhashable originals/keys

    for x in iterable:
        # First try to treat the ORIGINAL object as hashable — if that works,
        # we dedupe by the original value (so 'a' and 'A' remain distinct).
        try:
            if x in seen_hashable:
                continue
            seen_hashable.add(x)
            yield x
            continue
        except TypeError:
            pass  # x is unhashable; we'll use key (if provided) or fallback comparisons

        # x is unhashable
        k = x if key is None else key(x)

        # Try to dedupe using the key if it's hashable; otherwise use list membership
        try:
            if k in seen_hashable:
                continue
            seen_hashable.add(k)
            yield x
        except TypeError:
            if k in seen_unhashable:
                continue
            seen_unhashable.append(k)
            yield x

In [9]:
assert list(unique_everseen([1,1,2,2,3])) == [1,2,3]
assert list(unique_everseen(['a','A','a'], key=str.lower)) == ['a','A']
assert list(unique_everseen([[1],[1],[2]])) == [[1],[2]]  # unhashable lists
print('OK - 4')

OK - 4


## 5) Sliding window over an iterable
Implement `window(iterable, size)` yielding successive overlapping windows as tuples.
- Use a bounded `deque` to keep O(size) memory.
- Windows appear once they reach full size.

In [10]:
# TODO: implement window
from collections import deque

def window(iterable, size):
    if size <= 0:
        raise ValueError('size must be > 0')
    it = iter(iterable)
    d = deque(maxlen=size)
    for x in it:
        d.append(x)
        if len(d) == size:
            yield tuple(d)

list(window(range(5), 3))

[(0, 1, 2), (1, 2, 3), (2, 3, 4)]

In [11]:
assert list(window([1,2,3,4], 3)) == [(1,2,3),(2,3,4)]
assert list(window('abcd', 2)) == [('a','b'),('b','c'),('c','d')]
print('OK - 5')

OK - 5


## 6) Round-robin merge of multiple iterables
Implement `round_robin(*iterables)` yielding in rotation from each iterable until all are exhausted.
- Don’t pre-materialize inputs.
- Drop iterators as they finish (no empty yields).

In [12]:
# TODO: implement round_robin
from collections import deque

def round_robin(*iterables):
    q = deque(iter(it) for it in iterables)
    while q:
        it = q.popleft()
        try:
            item = next(it)
        except StopIteration:
            continue
        else:
            yield item
            q.append(it)

list(round_robin([1,2], 'ab', (10,)))

[1, 'a', 10, 2, 'b']

In [13]:
assert list(round_robin([1,2,3], ['a','b'], [10])) == [1,'a',10,2,'b',3]
assert list(round_robin([], [1,2])) == [1,2]
print('OK - 6')

OK - 6


## 7) Flatten arbitrarily nested iterables (but not strings/bytes)
Implement `flatten(nested)` that recursively yields scalars from nested structures.
- Treat `str`, `bytes`, `bytearray` as **atomic** (do not iterate into chars).

In [14]:
# TODO: implement flatten
def _is_nonstring_iterable(x):
    try:
        iter(x)
    except TypeError:
        return False
    else:
        return not isinstance(x, (str, bytes, bytearray))

def flatten(nested):
    for x in nested:
        if _is_nonstring_iterable(x):
            yield from flatten(x)
        else:
            yield x

list(flatten([1,[2,[3,'ab']],('x','y')]))

[1, 2, 3, 'ab', 'x', 'y']

In [15]:
assert list(flatten([1,[2,[3,'ab']],('x','y')])) == [1,2,3,'ab','x','y']
assert list(flatten(((1,2),[3, (4,5)]))) == [1,2,3,4,5]
print('OK - 7')

OK - 7


## 8) Make a rewindable wrapper around a one-shot iterator
Implement `Rewindable(iterator)` class:
- Iterating once consumes the source and buffers elements.
- Subsequent iterations replay from the buffer (no re-reading source).
- Provide `.reset()` to manually rewind.
- Must implement standard protocol: `__iter__`, `__next__`.

In [16]:
# TODO: implement Rewindable
class Rewindable:
    def __init__(self, iterator):
        self._source = iter(iterator)
        self._buffer = []
        self._index = 0

    def __iter__(self):
        # When iteration starts, replay from beginning of buffer
        self._index = 0
        return self

    def __next__(self):
        if self._index < len(self._buffer):
            val = self._buffer[self._index]
            self._index += 1
            return val
        # Pull new item from source, buffer it
        val = next(self._source)
        self._buffer.append(val)
        self._index += 1
        return val

    def reset(self):
        self._index = 0

r = Rewindable(iter([1,2,3]))
list(r)

[1, 2, 3]

In [17]:
r = Rewindable((x for x in [1,2,3]))
assert list(r) == [1,2,3]
assert list(r) == [1,2,3]  # replays from buffer
r.reset()
assert next(iter(r)) == 1
print('OK - 8')

OK - 8


## 9) Seek forward *n* steps in an iterator
Implement `seek_iter(iterator, n)` that advances an iterator by `n` items and returns `(last_discarded, remaining_iterator)`.
- If fewer than `n` items exist, return `(None, remaining_iterator)` (already exhausted).

In [18]:
# TODO: implement seek_iter
def seek_iter(iterator, n):
    it = iter(iterator)
    last = None
    for _ in range(max(0, int(n))):
        try:
            last = next(it)
        except StopIteration:
            return None, it
    return last, it

seek_iter(iter(range(5)), 3)

(2, <range_iterator at 0x2d335534ff0>)

In [19]:
last, it = seek_iter(iter([10,20,30,40]), 2)
assert last == 20 and list(it) == [30,40]
last2, it2 = seek_iter(iter([1,2]), 5)
assert last2 is None and list(it2) == []
print('OK - 9')

OK - 9


## 10) Peek the first element without losing it
Implement `peek(it)` returning `(first, new_iterator)` where `new_iterator` yields `first` and then the rest of `it`.
- If `it` is empty, raise `ValueError('empty iterator')`.
- Don’t pre-materialize all items (no `list(it)`).

In [20]:
# TODO: implement peek
def peek(it):
    it = iter(it)
    try:
        first = next(it)
    except StopIteration:
        raise ValueError('empty iterator')
    def new_iter():
        yield first
        yield from it
    return first, new_iter()

peek(iter([9,8,7]))

(9, <generator object peek.<locals>.new_iter at 0x000002D3355B9FF0>)

In [21]:
first, rest = peek(iter([9,8,7]))
assert first == 9 and list(rest) == [9,8,7]
try:
    peek(iter([]))
except ValueError as e:
    assert 'empty' in str(e)
print('OK - 10')

OK - 10
